In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import IsolationForest
import pickle
from joblib import dump, load

# Read the Excel file into a DataFrame
data = pd.read_excel(r"C:\Users\SHESHAADHRI\OneDrive - vit.ac.in\GG\ENERGY.xlsx")

# Drop irrelevant columns and datetime columns
data.drop(['UserID', 'Timestamp'], axis=1, inplace=True)

# Handle missing values
data.dropna(inplace=True)

# One-hot encode categorical variables
data = pd.get_dummies(data, columns=['EngineType', 'EnergySource'])

# Convert RecordID to string
data['RecordID'] = data['RecordID'].astype(str)

# Split data into features (X) and target (y)
X = data.drop(['RecordID', 'AmountConsumed'], axis=1)  # Features
y = data['AmountConsumed']  # Target

# Select only numeric columns
X_numeric = X.select_dtypes(include=['number'])

# Ensure all columns in X_numeric are numeric
X_numeric = X_numeric.astype(float)

# Save feature names
feature_names = X_numeric.columns.tolist()
with open('feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

# Train anomaly detection model
anomaly_model = IsolationForest(contamination=0.1)  # Adjust contamination based on your data
anomaly_model.fit(X_numeric)

# Save the trained model using joblib
dump(anomaly_model, 'anomaly_model.joblib')

# Calculate similarity matrix based on location and engine type
def calculate_similarity_matrix(data):
    # Select features for similarity calculation
    location_columns = [col for col in data.columns if col.startswith('Location_')]
    engine_type_columns = [col for col in data.columns if col.startswith('EngineType_')]
    features = data[location_columns + engine_type_columns]
    
    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(features, features)
    return similarity_matrix

def recommend_solutions(record_id, similarity_matrix, data):
    # Find index of given record ID
    record_index = data[data['RecordID'] == record_id].index[0]
    
    # Get location of the given record
    location = data.loc[record_index, 'Location']
    
    # Check if 'EnergySource' column exists in the DataFrame
    if 'EnergySource' in data.columns:
        # Get engine source of the given record
        engine_source = data.loc[record_index, 'EnergySource']
    else:
        engine_source = 'Not available'
    
    # Get records with the same location but different engine source and less consumption
    similar_records = data[(data['Location'] == location) &
                           (data['AmountConsumed'] < data.loc[record_index, 'AmountConsumed'])]
    
    # Find the best solution based on least amount consumed and least CO2 emissions if tied
    best_solution = similar_records.sort_values(by=['AmountConsumed', 'CO2Emissions (kg)']).iloc[0]
    
    return best_solution

# Calculate similarity matrix
similarity_matrix = calculate_similarity_matrix(data)

# Detect anomalies
anomaly_mask = anomaly_model.predict(X_numeric) == -1
anomaly_records = data[anomaly_mask]

# Recommend solutions for each anomaly
for record_id in anomaly_records['RecordID']:
    print(f"Solutions for anomaly record ID '{record_id}':")
    best_solution = recommend_solutions(record_id, similarity_matrix, data)
    print(f"RecordID: {best_solution['RecordID']}, Location: {best_solution['Location']}, EngineType: {best_solution.get('EngineType', 'Not available')}, AmountConsumed: {best_solution['AmountConsumed']}")


Solutions for anomaly record ID 'r17':
RecordID: r8, Location: L2, EngineType: Not available, AmountConsumed: 42.0
Solutions for anomaly record ID 'r19':
RecordID: r9, Location: L1, EngineType: Not available, AmountConsumed: 12.0
Solutions for anomaly record ID 'r31':
RecordID: r8, Location: L2, EngineType: Not available, AmountConsumed: 42.0
